In [1]:
# import packages
# matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

In [2]:
#set random seed and percentage of test data
RANDOM_SEED = 314 #used to help randomly select the data points
TEST_PCT = 0.2 # 20% of the data


In [3]:
df = pd.read_csv("creditcard.csv") 
df.head(n=5) 

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.shape #secondary check on the size of the dataframe

(284807, 31)

In [5]:
df.isnull().values.any() #check to see if any values are null, which there are not

False

In [6]:
pd.value_counts(df['Class'], sort = True) #class comparison 0=Normal 1=Fraud

0    284315
1       492
Name: Class, dtype: int64

In [7]:
normal_df = df[df.Class == 0] #save normal_df observations into a separate df
fraud_df = df[df.Class == 1] #do the same for frauds

In [8]:
normal_df.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [9]:
fraud_df.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [10]:
df['Class'].unique() # 0 = no fraud, 1 = fraudulent

array([0, 1], dtype=int64)

In [11]:
#Normalize and Scale Data
#data = df.drop(['Time'], axis=1) #if you think the var is unimportant
df_norm = df
df_norm['Time'] = StandardScaler().fit_transform(df_norm['Time'].values.reshape(-1, 1))
df_norm['Amount'] = StandardScaler().fit_transform(df_norm['Amount'].values.reshape(-1, 1))

In [12]:
#Dividing Training and Test Set
train_x, test_x = train_test_split(df_norm, test_size=TEST_PCT, random_state=RANDOM_SEED)
train_x = train_x[train_x.Class == 0] #where normal transactions
train_x = train_x.drop(['Class'], axis=1) #drop the class column


test_y = test_x['Class'] #save the class column for the test set
test_x = test_x.drop(['Class'], axis=1) #drop the class column

train_x = train_x.values #transform to ndarray
test_x = test_x.values

In [13]:
train_x.shape

(227468, 30)

In [14]:
#Creating the Model
nb_epoch = 50
batch_size = 128
input_dim = train_x.shape[1] #num of columns, 30
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [15]:
#Model Training and Logging
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(train_x, train_x,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_x, test_x),
                    verbose=1,
                    callbacks=[cp, tb]).history

Epoch 1/50
1778/1778 [==============================] - 6s 3ms/step - loss: 0.9318 - accuracy: 0.4074 - val_loss: 0.8347 - val_accuracy: 0.5783
Epoch 2/50
1778/1778 [==============================] - 3s 2ms/step - loss: 0.7909 - accuracy: 0.5932 - val_loss: 0.7998 - val_accuracy: 0.6285
Epoch 3/50
1778/1778 [==============================] - 4s 2ms/step - loss: 0.7614 - accuracy: 0.6246 - val_loss: 0.7875 - val_accuracy: 0.6353
Epoch 4/50
1778/1778 [==============================] - 3s 2ms/step - loss: 0.7508 - accuracy: 0.6371 - val_loss: 0.7800 - val_accuracy: 0.6438
Epoch 5/50
1778/1778 [==============================] - 3s 2ms/step - loss: 0.7521 - accuracy: 0.6450 - val_loss: 0.7702 - val_accuracy: 0.6578
Epoch 6/50
1778/1778 [==============================] - 3s 2ms/step - loss: 0.7310 - accuracy: 0.6579 - val_loss: 0.7621 - val_accuracy: 0.6694
Epoch 7/50
1778/1778 [==============================] - 3s 2ms/step - loss: 0.7346 - accuracy: 0.6677 - val_loss: 0.7576 - val_accuracy:

In [16]:
autoencoder = load_model('autoencoder_fraud.h5')

In [17]:
#Reconstruction Error Check
test_x_predictions = autoencoder.predict(test_x)
mse = np.mean(np.power(test_x - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_y})
error_df.describe()

,Reconstruction_error,True_class
count,56962.000000,56962.000000
mean,0.734975,0.002019
std,3.165660,0.044887
min,0.045243,0.000000
25%,0.252267,0.000000
50%,0.403755,0.000000
75%,0.638371,0.000000
max,190.810488,1.000000


In [18]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [20]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [22]:
clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=30, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [23]:
clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                496       
_________________________________________________________________
dense_5 (Dense)              (None, 18)                306       
_________________________________________________________________
dropout (Dropout)            (None, 18)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)                380       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 25        
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
______________________________________________________

In [24]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
clf.fit(X_train, Y_train, batch_size=15, epochs=2)

Epoch 1/2
17089/17089 [==============================] - 29s 2ms/step - loss: 0.0227 - accuracy: 0.9984
Epoch 2/2
17089/17089 [==============================] - 27s 2ms/step - loss: 0.0032 - accuracy: 0.9994


In [26]:
score = clf.evaluate(X_test, Y_test, batch_size=128)
print('\nAnd the Score is ', score[1] * 100, '%')

223/223 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 0.9992

And the Score is  99.91924166679382 %
